In [0]:
!pip install gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

In [0]:
# 認証系の処理
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# スプレッドシートの1枚めを開く
worksheet = gc.open('pythongram_sample').sheet1


In [0]:
rows = worksheet.get_all_values()
import pandas as pd

# 0行目にヘッダを読み込んでしまうのでちょっと工夫
df = pd.DataFrame.from_records(rows)
# header = df[:0]
col_list = df.values[0].tolist()
df = df.drop(0)
df.columns = col_list
df

In [0]:
# 国が中国のものと空のものは取り除く
df_without_cn = df[(df['country_code'] != 'CN') & (df['country_code'] != '')]
df_without_cn

In [0]:
grouped =df_without_cn.groupby(['username','country_code'])
grpuped_df = grouped.size().to_frame('size')
grpuped_df


In [0]:
# 各usernameごとに国別のデータ数が一番多いものを抽出する
idx = grpuped_df.groupby('username')['size'].idxmax()

df_max_count_country =  grpuped_df.loc[idx]
df_max_count_country

In [0]:
# スプレッドシートの３枚めを開く
worksheet3 = gc.open('pythongram_sample').get_worksheet(2)

i = 2
for index, row in df_max_count_country.iterrows():
  cell_list = worksheet3.range('A' + str(i) + ':' + 'C' + str(i))
  print('A' + str(i) + ':' + 'C' + str(i))
  #   print(index[0])
  #   print(index[1])
  #   print(row)
  cell_list[0].value = index[0]
  cell_list[1].value = index[1]
  cell_list[2].value = row.values[0].item()
  print(type(row.values[0].item()))
  worksheet3.update_cells(cell_list)
  i += 1
    

In [0]:
list = [] 
for name, group in grouped:
  #   print(name)
  list.append(group.head(1).values[0].tolist())

#  このリストでユーザ名 x 国名の1つ目のデータのリストが格納されている
list

In [0]:
# スプレッドシートの2枚めを開く。ただしスプレッドシートをGoogle mapのマイマップに展開するには、スプレッドシートの１枚目のシートにコピーする必要がある
worksheet2 = gc.open('pythongram_sample').get_worksheet(1)

i = 2
for v in list:
   print('A' + str(i) + ':' + 'G' + str(i))
  cell_list = worksheet2.range('A' + str(i) + ':' + 'G' + str(i))
  cell_list[0].value = v[0]
  cell_list[1].value = v[1]
  cell_list[2].value = v[2]
  cell_list[3].value = v[3]
  cell_list[4].value = v[4]
  cell_list[5].value = v[5]
  cell_list[6].value = v[6]

  worksheet2.update_cells(cell_list)
  i += 1
